In [1]:
import requests
from openai import OpenAI
from dotenv import load_dotenv
load_dotenv()
import json

client = OpenAI()
s = requests.Session()

GROUP_THEM_URL = "http://127.0.0.1:8080/groupthem"

### WordGuess

In [9]:
todo

{'guesses': [], 'maximum_guesses': 6}

### GroupThem

In [2]:
state = s.get(GROUP_THEM_URL).json()
print(state)
schema = s.get(GROUP_THEM_URL + "/schema").json()
print(schema)

{'guesses': [], 'available_words': ['cake', 'pie', 'pudding', 'cookie', 'pen', 'notebook', 'stapler', 'envelope', 'holmes', 'poirot', 'marple', 'spade', 'centaur', 'mermaid', 'minotaur', 'sphinx']}
{'$schema': 'http://json-schema.org/draft-07/schema#', 'title': 'GroupThemRequest', 'type': 'object', 'required': ['guess'], 'properties': {'guess': {'type': 'array', 'items': {'type': 'string'}}}}


In [60]:
schema = "{reasoning: String, connections: [String; 4]}"

conversation = [
    {"role": "system", "content": "You are a helpful assistant. Create a json object with the connection groups & your reasoning. ### Schema\n" + schema },
    {"role": "user", "content": "Create the mentioned JSON object through trying to find a connection between exactly 4 of these items: " + ", ".join(state["available_words"]) + "."},
]

for _ in range(10):
    # Ask model to attempt connection
    response = client.chat.completions.create(
        model="gpt-4o",
        messages=conversation
    ).choices[0].message.content
    
    start = response.find("{")
    end = response.find("}", start)
    json_response = json.loads(response[start:end+1])
    print(json_response)
    # Send connections
    payload = {"guess": json_response["connections"]}
    
    try:
        available_words = s.post(GROUP_THEM_URL, json=payload).json()["available_words"]
    except Exception as e:
        print(e)
        break
    
    win = json_response["connections"][0] not in available_words
    if len(available_words) == 0:
        print("You won!")
        break
    else:
        conversation.append({"role": "user", "content": f"You were {'CORRECT!!' if win else 'INCORRECT'}. \n\n### Current List: " + ", ".join(available_words) + "."})
        

{'reasoning': 'The connection between the selected items is that they are all mythical creatures often found in folklore and mythology.', 'connections': ['centaur', 'mermaid', 'minotaur', 'sphinx']}
{'reasoning': "The connection group identified consists of 'holmes', 'poirot', 'marple', and 'spade'. These items are all renowned fictional detectives, characters in mystery novels and stories.", 'connections': ['holmes', 'poirot', 'marple', 'spade']}
{'reasoning': 'The connection between the selected items is that they are all office supplies.', 'connections': ['pen', 'notebook', 'stapler', 'envelope']}
{'reasoning': 'These four items are all types of desserts or sweet treats.', 'connections': ['cake', 'pie', 'pudding', 'cookie']}
You won!
